# Lecture 9: DNFs & Random Restrictions

**Topics**: DNF formulas, Random restrictions, Switching Lemma preview

**O'Donnell Chapters**: 3.3, 4.1, 4.3

---

## Key Concepts

1. **DNF**: Disjunctive Normal Form (OR of ANDs)
2. **Random Restriction**: Fix random subset of variables
3. **Switching Lemma**: DNFs simplify under restrictions

In [ ]:
import numpy as np
import boolfunc as bf

# Tribes function is a canonical DNF
tribes = bf.tribes(3, 9)  # 3 tribes of 3 variables each
print(f"Tribes(3,9): {tribes.n_vars} variables")

# Direct API methods
print(f"Total Influence: {tribes.total_influence():.4f}")
print(f"Spectral concentration on degree ≤2: {tribes.spectral_concentration(2):.4f}")

## Summary

- **DNFs** have Fourier weight concentrated on low degrees (Mansour's Theorem)
- **Random restrictions** simplify DNFs dramatically (Switching Lemma)
- **Tribes** is the canonical example of a balanced DNF

```python
tribes = bf.tribes(k, n)  # k tribes, each AND of n/k variables
```